In [2]:
from utils import *

import os

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [205]:
# Misc
LOG_DIR = './log_pt/'

# Training Parameters
learning_rate = 0.001
num_epochs = 300
batch_size = 256
display_step = 10

# Network Parameters
num_classes = 6
keep_rate = 1

repeat_size = 82
crop_size = 80

In [206]:
raw = pd.read_csv('./vultures.csv', header=None)

In [207]:
mean = np.array([0. for i in  range(3)])
X = []
Y = []

for i in range(len(raw)):
    
    row = np.array(raw.loc[i][~raw.loc[i].isnull()])
    data = row[0:-1].astype(float)
    label = row[-1]
    
    x_axis = data[0::3]
    y_axis = data[1::3]
    z_axis = data[2::3]
    
    x_axis = repeat_crop_data (x_axis, repeat_size)
    y_axis = repeat_crop_data (y_axis, repeat_size)
    z_axis = repeat_crop_data (z_axis, repeat_size)
    
    mean[0] += x_axis.mean()
    mean[1] += y_axis.mean()
    mean[2] += z_axis.mean()
    
    X.append(np.stack((x_axis, y_axis, z_axis)))
    Y.append(label)

mean = mean/len(raw)
    
X = np.array(X).transpose((0, 2, 1))
Y = np.array(Y)
label_names = np.unique(Y)

le = preprocessing.LabelEncoder()
le.fit(np.unique(Y))
Y = le.transform(Y)

In [208]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

In [209]:
class acc_dataset_augment (torch.utils.data.Dataset):
    
    def __init__ (self, X, Y, mean, input_size, crop_size):
        super().__init__()
        self.X = X
        self.Y = Y.astype(np.float)
        self.mean = mean
        self.input_size = input_size
        self.crop_size = crop_size
        
    def __len__ (self):
        return len(self.X)
    
    def __getitem__ (self, idx):
        
        x = self.X[idx]
        y = self.Y[idx]
        
        x = (x - self.mean).transpose(1, 0)
        
        start_idx = np.random.randint(self.input_size - self.crop_size, size=1).item()
        
        x_axis = x[0, start_idx:start_idx+self.crop_size]
        y_axis = x[1, start_idx:start_idx+self.crop_size]
        z_axis = x[2, start_idx:start_idx+self.crop_size]
        
        x = np.stack((x_axis, y_axis, z_axis))
            
        return torch.tensor(x), torch.tensor(y).type(torch.LongTensor)

In [210]:
train_set = acc_dataset_augment(train_x, train_y, mean, input_size=repeat_size, crop_size=crop_size)
test_set = acc_dataset_augment(test_x, test_y, mean, input_size=repeat_size, crop_size=crop_size)

In [211]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [212]:
class MyNet (nn.Module):
    
    def __init__ (self, num_classes, input_dim, drop_rate):
        super().__init__()
        
        self.input_dim = input_dim
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc3 = nn.Linear(self.input_dim*64, 1024)
        self.drop = nn.Dropout(p=drop_rate)
        self.fc4 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        #self.pretrained_resnet.avgpool(x)
        x = x.view(-1, self.input_dim*64)
        
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = self.fc4(x)

        return x

In [213]:
net = MyNet(num_classes=num_classes, input_dim=crop_size, drop_rate=0.3).double()

In [214]:
net.to(device)

MyNet(
  (conv1): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc3): Linear(in_features=5120, out_features=1024, bias=True)
  (drop): Dropout(p=0.3)
  (fc4): Linear(in_features=1024, out_features=6, bias=True)
)

In [216]:
# Loss function
loss_function = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 30 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [217]:
step = 0

net.train()

for epoch in range(num_epochs):
    
    exp_lr_scheduler.step()
    
    for data in train_loader:
        
        x, y = data
        
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        logits = net(x)
        
        loss = loss_function(logits, y)
        
        loss.backward()
        
        optimizer.step()
        
        step += 1
        
        _, preds = torch.max(logits, 1)
        
        correct = (preds == y).sum().item()
        
        accuracy = correct/len(x)
        
        if step % display_step == 0:
            print('Epoch: %d, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (epoch, step, loss, accuracy))
            
print('Final, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (step, loss, accuracy))

Epoch: 4, Step: 10, Minibatch loss: 0.995, Minibatch accuracy: 0.661
Epoch: 9, Step: 20, Minibatch loss: 0.821, Minibatch accuracy: 0.694
Epoch: 14, Step: 30, Minibatch loss: 0.569, Minibatch accuracy: 0.809
Epoch: 19, Step: 40, Minibatch loss: 0.420, Minibatch accuracy: 0.836
Epoch: 24, Step: 50, Minibatch loss: 0.275, Minibatch accuracy: 0.929
Epoch: 29, Step: 60, Minibatch loss: 0.204, Minibatch accuracy: 0.951
Epoch: 34, Step: 70, Minibatch loss: 0.236, Minibatch accuracy: 0.934
Epoch: 39, Step: 80, Minibatch loss: 0.148, Minibatch accuracy: 0.967
Epoch: 44, Step: 90, Minibatch loss: 0.180, Minibatch accuracy: 0.934
Epoch: 49, Step: 100, Minibatch loss: 0.207, Minibatch accuracy: 0.940
Epoch: 54, Step: 110, Minibatch loss: 0.161, Minibatch accuracy: 0.951
Epoch: 59, Step: 120, Minibatch loss: 0.150, Minibatch accuracy: 0.967
Epoch: 64, Step: 130, Minibatch loss: 0.155, Minibatch accuracy: 0.962
Epoch: 69, Step: 140, Minibatch loss: 0.105, Minibatch accuracy: 0.978
Epoch: 74, Step: 

In [219]:
net.eval()

test_correct = 0

with torch.no_grad():
    
    for data in test_loader:

        test_x, test_y = data
        
        test_x, test_y = test_x.to(device), test_y.to(device)

        test_logits = net(test_x)
        _, test_preds = torch.max(test_logits, 1)

        test_correct_batch = (test_preds == test_y).sum().item()

        test_correct += test_correct_batch

print('Test accuracy: %.3f' % (test_correct / len(test_set)))

Test accuracy: 0.735


In [189]:
torch.save(net.state_dict(), os.path.join(LOG_DIR, 'model_%s_checkpoint.pth.tar' % 'vulter_aug_ep_50_step_10'))

In [60]:
# Training Parameters
learning_rate = 0.01
num_epochs = 50

In [ ]:
print('Load pretrained ResNet.')

# Load pretrained model
if CAFFE_WEIGHTS:
    print('Model using caffe pretrained weights.')
    resnet = torchvision.models.resnet152(pretrained=False)
    weights = torch.load('./caffe_weights/resnet152.pth')
    resnet.load_state_dict(weights)
else:
    print('Model using pytorch pretrained weights.')
    resnet = torchvision.models.resnet152(pretrained=True)

# Freeze all layers
for param in resnet.parameters():
    param.requires_grad = False

print('Done.')    


if DROPOUT:
    print('Model using dropout.')
    resnet = MyResNet(resnet, NUM_CLASSES, DROPOUT_RATE, FC_ADD_DIM)
    # Optimizer only on the last fc layers
    optimizer = optim.SGD(list(resnet.fc.parameters()) + list(resnet.fc_add.parameters()), lr=LEARNING_RATE, momentum=MOMENTUM)
else:
    print('Model not using dropout.')
    # Reset the fc layer
    num_features = resnet.fc.in_features
    resnet.fc = nn.Linear(num_features, NUM_CLASSES)
    # Optimizer only on the last fc layers
    optimizer = optim.SGD(list(resnet.fc.parameters()), lr=LEARNING_RATE, momentum=MOMENTUM)


In [23]:
checkpoint = torch.load('./log_pt/model_storks_plain_ep_60_step_10_checkpoint.pth.tar')

ft_net = MyNet(num_classes=num_classes, input_dim=repeat_size).double()

ft_net.load_state_dict(checkpoint)

In [24]:
ft_net.fc4 = nn.Linear(1024, num_classes)

In [25]:
ft_net = ft_net.to(device)

In [26]:
optimizer = optim.Adam(ft_net.parameters(), lr=learning_rate)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [27]:
step = 0

ft_net.train()

for epoch in range(num_epochs):
    
    exp_lr_scheduler.step()
    
    for data in train_loader:
        
        x, y = data
        
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        logits = ft_net(x)
        
        loss = loss_function(logits, y)
        
        loss.backward()
        
        optimizer.step()
        
        step += 1
        
        _, preds = torch.max(logits, 1)
        
        correct = (preds == y).sum().item()
        
        accuracy = correct/len(x)
        
        if step % display_step == 0:
            print('Epoch: %d, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (epoch, step, loss, accuracy))
            
print('Final, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (step, loss, accuracy))

RuntimeError: Expected object of type torch.cuda.FloatTensor but found type torch.cuda.DoubleTensor for argument #4 'mat1'

In [ ]:
ft_net.eval()

test_correct = 0

with torch.no_grad():
    
    for data in test_loader:

        test_x, test_y = data

        test_logits = ft_net(test_x)
        _, test_preds = torch.max(test_logits, 1)

        test_correct_batch = (test_preds == test_y).sum().item()

        test_correct += test_correct_batch

print('Test accuracy: %.3f' % (test_correct / len(test_set)))